In [31]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize
import pandas as pd

In [2]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = "/Users/wzhang/Documents/WorkFile/GoogleAPI/magnetic-hawk-523-d29583f37b64.json"  #Service Accounts
VIEW_ID = 'ga:23787976' # ga:121755352 for INSIDER

In [3]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(
  KEY_FILE_LOCATION, SCOPES)

In [4]:
analytics = build('analyticsreporting', 'v4', credentials=credentials)

In [7]:
response = analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '2020-01-07', 'endDate': 'yesterday'}],
          'metrics': [{'expression': 'ga:sessions'}],
          'dimensions': [{'name': 'ga:country'}]
        }]
      }
  ).execute()

In [34]:
response.get

<function dict.get(key, default=None, /)>

In [35]:
def print_response(response):
  list = []
  # get report data
  for report in response.get('reports', []):
    # set column headers
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
          dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
          for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
            if ',' in value or '.' in value:
              dict[metric.get('name')] = float(value)
            else:
              dict[metric.get('name')] = int(value)

        list.append(dict)
    
    df = pd.DataFrame(list)
    return df

In [36]:
print_response(response)

,ga:dimension9,ga:pageviews
0,-marissa-mayer-cv-2017-3,9
1,-mavs-owner-mark-cuban-billionaire-lifestyle-2...,83
2,(null)/50-billion-de-shaw-composite-fund-posts...,26
3,(null)/accenture-revealed-latest-cohort-of-its...,11
4,(null)/aG93LXRvLX,2
5,(null)/airbnb-softwarotoutpeoplewith%27narciss...,10
6,(null)/dHJ1bXAtdG,2
7,(null)/dWstZmludG,2
8,(null)/elon-musk-just-dad-danced-at-teslas-new...,7
9,(null)/exclusive-impossible-food-abandons-purs...,7


In [11]:
def print_response(response):
  """Parses and prints the Analytics Reporting API V4 response.

  Args:
    response: An Analytics Reporting API V4 response.
  """
  for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    for row in report.get('data', {}).get('rows', []):
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])
        
        for header, dimension in zip(dimensionHeaders, dimensions):
            print(header + ': ' + dimension)

        for i, values in enumerate(dateRangeValues):
            print('Date range: ' + str(i))
            for metricHeader, value in zip(metricHeaders, values.get('values')):
                print(metricHeader.get('name') + ': ' + value)

In [12]:
print_response(response)

ga:dimension9: -marissa-mayer-cv-2017-3
Date range: 0
ga:pageviews: 9
ga:dimension9: -mavs-owner-mark-cuban-billionaire-lifestyle-2014-8
Date range: 0
ga:pageviews: 83
ga:dimension9: (null)/50-billion-de-shaw-composite-fund-posts-105-return-2019-2020-1)
Date range: 0
ga:pageviews: 26
ga:dimension9: (null)/accenture-revealed-latest-cohort-of-its-fintech-accelerator-2020-1)
Date range: 0
ga:pageviews: 11
ga:dimension9: (null)/aG93LXRvLX
Date range: 0
ga:pageviews: 2
ga:dimension9: (null)/airbnb-softwarotoutpeoplewith%27narcissismorpsychopathy%27e-predicts-if-guests-are-psychopaths-patent-2020-1
Date range: 0
ga:pageviews: 10
ga:dimension9: (null)/dHJ1bXAtdG
Date range: 0
ga:pageviews: 2
ga:dimension9: (null)/dWstZmludG
Date range: 0
ga:pageviews: 2
ga:dimension9: (null)/elon-musk-just-dad-danced-at-teslas-new-shanghai-Gigafactory-2020-1
Date range: 0
ga:pageviews: 7
ga:dimension9: (null)/exclusive-impossible-food-abandons-pursuit-of-mcdonalds-burger-deal-2020-1
Date range: 0
ga:pageviews

ga:pageviews: 37
ga:dimension9: 100-year-starship-plan-for-interstellar-travel-2016-3
Date range: 0
ga:pageviews: 3
ga:dimension9: 100-years-of-american-mens-fashion-in-pictures-2015-7
Date range: 0
ga:pageviews: 48
ga:dimension9: 100-years-of-mens-beauty-trends-2015-11
Date range: 0
ga:pageviews: 108
ga:dimension9: 100-years-of-real-women-2015-11
Date range: 0
ga:pageviews: 16
ga:dimension9: 1000-slices-of-cheese-on-your-bk-whopper-2013-7
Date range: 0
ga:pageviews: 2
ga:dimension9: 1000-us-troops-chaotic-demoralizing-retreat-syria-2019-10
Date range: 0
ga:pageviews: 5
ga:dimension9: 10000-pixel-planetary-nebula-collage-2014-10
Date range: 0
ga:pageviews: 3
ga:dimension9: 100000-salary-around-the-world-2011-4
Date range: 0
ga:pageviews: 112
ga:dimension9: 101-best-inspirational-quotes-for-entrepreneurs-2013-9
Date range: 0
ga:pageviews: 5
ga:dimension9: 101-businesses-you-can-start-with-less-than-100-2016-10
Date range: 0
ga:pageviews: 2
ga:dimension9: 101st-airborne-members-share-tie

Date range: 0
ga:pageviews: 22
ga:dimension9: 15-poignant-photos-that-show-what-life-was-like-during-the-vietnam-war-2015-4
Date range: 0
ga:pageviews: 371
ga:dimension9: 15-polarizing-food-debates-2016-3
Date range: 0
ga:pageviews: 560
ga:dimension9: 15-politicians-on-twitter-you-must-follow-2013-7
Date range: 0
ga:pageviews: 58
ga:dimension9: 15-powerful-benefits-of-living-in-the-moment-2014-3
Date range: 0
ga:pageviews: 9
ga:dimension9: 15-quotes-about-creativity-2014-5
Date range: 0
ga:pageviews: 6
ga:dimension9: 15-racist-brand-mascots-and-logos-2014-6
Date range: 0
ga:pageviews: 640
ga:dimension9: 15-reasons-why-waterbeds-are-for-suckers-2012-12
Date range: 0
ga:pageviews: 33
ga:dimension9: 15-resource-rich-countries-2010-4
Date range: 0
ga:pageviews: 337
ga:dimension9: 15-ridiculous-trademarking-attempts-2011-4
Date range: 0
ga:pageviews: 177
ga:dimension9: 15-rising-star-employees-at-sap-2015-11
Date range: 0
ga:pageviews: 16
ga:dimension9: 15-roles-every-startup-needs-filled-2